In [ ]:
import json
import pandas as pd
import plotly.graph_objects as go

pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100

# Load raw data

In [ ]:
with open('raw/disco_elysium.json', 'r') as f:
    data = json.load(f)

# Extract info

In [ ]:
actors = data['actors']
items = data['items']
variables = data['variables']
conversations = data['conversations']
syncInfo = data['syncInfo']


In [ ]:
def convert_actors(actors):
    fixed_actors = {}
    needed_fields = [
        'Name', 'Description',
        'character_short_name', 'IsNPC',
        'short_description', 'LongDescription',
    ]
    
    for act in actors:
        idx = act['id']
        fixed_actors[idx] = {}
        for field in act['fields']:
            if field['title'] in needed_fields:
                fixed_actors[idx][field['title']] = field['value']
    return fixed_actors

def convert_dialog_entr(conversations: list):
    fixed_dialog = {}
    needed_fields = [
        'Title', 'Dialogue Text',
        'Actor', 'Conversant',
        #'InputId', 'OutputId',
    ]
    for conversation in conversations:
        for line in conversation['dialogueEntries']:
            idx = (line['conversationID'], line['id'])
            fixed_dialog[idx] = {}
            for field in line['fields']:
                if field['title'] in needed_fields:
                    fixed_dialog[idx][field['title']] = field['value']

            fixed_dialog[idx]['outgoingLinks'] = []
            for link in line['outgoingLinks']:
                fixed_dialog[idx]['outgoingLinks'].append(
                    (link['destinationConversationID'], link['destinationDialogueID']))
            fixed_dialog[idx]['isGroup'] = line['isGroup']
            fixed_dialog[idx]['canvasRect_x'] = line['canvasRect']['x']
            fixed_dialog[idx]['canvasRect_y'] = line['canvasRect']['y']
            fixed_dialog[idx]['canvasRect_width'] = line['canvasRect']['width']
            fixed_dialog[idx]['canvasRect_height'] = line['canvasRect']['height']
    return fixed_dialog
        

In [ ]:
actor_table = pd.DataFrame(convert_actors(actors)).T
dia_table = pd.DataFrame(convert_dialog_entr(conversations)).T

In [ ]:
dia_table[dia_table['Dialogue Text'].isna() == False]['Dialogue Text'].apply(lambda x: len(x.split(' '))).sum()

In [ ]:
actor_table.to_csv('table/actor.csv')
dia_table.to_csv('table/convs.csv')

# Plotting stuff

In [ ]:
idx = 39
canva = dia_table.loc[idx].loc[:, ['canvasRect_x', 'canvasRect_y', 'outgoingLinks', 'Dialogue Text']]

In [ ]:
def plot_conv(dia_table, inp_conv_id):
    #idx = 100
    canva = dia_table.loc[inp_conv_id].loc[:, ['canvasRect_x', 'canvasRect_y', 'outgoingLinks', 'Dialogue Text']]

    colors = ['#872341', '#2d98b5']
    dot_color = '#BE3144'
    bg_color = '#22092C'

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=canva['canvasRect_x'],
        y=canva['canvasRect_y'],
        text=canva.index,#['Dialogue Text'],
        mode="markers",
        marker=dict(color=dot_color, )#size=2)
    ))

    for self_dia_id, other in canva.iterrows():
        
        x0 = other['canvasRect_x']
        y0 = other['canvasRect_y']
        links = other['outgoingLinks']
        for conv_id, dia_id in links:
            if conv_id != inp_conv_id:
                continue

            x2, y2 = canva.loc[dia_id][['canvasRect_x', 'canvasRect_y']]
            x1, y1 = (x0 + x2)/2, (y0 + y2)/2
            
            fig.add_trace(go.Scatter(
                x=[x0, x1], y=[y0, y1], showlegend=False, hoverinfo='none',
                mode="lines", line=dict(width=0.5, color=colors[0])))
            
            fig.add_trace(go.Scatter(
                x=[x1, x2], y=[y1, y2], showlegend=False,  hoverinfo='none',
                mode="lines", line=dict(width=0.5, color=colors[1])))

    fig.update_layout(
        autosize=False,
        width=1500,
        height=1000,
        margin=dict(l=0, r=0, b=0, t=0, pad=0),
        plot_bgcolor=bg_color,
        paper_bgcolor=bg_color,
    )

    fig.update_shapes(dict(xref='x', yref='y'))
    return fig

In [ ]:
def plot_3d_conv(dia_table, from_idx, to_idx):
    #idx = 100
    canva = dia_table.loc[from_idx:to_idx].loc[:, ['canvasRect_x', 'canvasRect_y', 'outgoingLinks', 'Dialogue Text']]

    colors = ['#872341', '#2d98b5']
    dot_color = '#BE3144'
    bg_color = '#22092C'

    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=canva['canvasRect_x'],
        y=canva['canvasRect_y'],
        z=canva.index.get_level_values(0),
        text=canva['Dialogue Text'],
        mode="markers",
        marker=dict(color=dot_color, size=2)
    ))

    for (self_conv_id, self_dia_id), other in canva.iterrows():
        
        x0 = other['canvasRect_x']
        y0 = other['canvasRect_y']
        z0 = self_conv_id
        links = other['outgoingLinks']
        for conv_id, dia_id in links:
            if conv_id not in canva.index:
                continue
            z2 = conv_id
            x2, y2 = canva.loc[conv_id, dia_id][['canvasRect_x', 'canvasRect_y']]
            
            x1, y1, z1 = (x0 + x2)/2, (y0 + y2)/2, (z0 + z2)/2
            fig.add_trace(go.Scatter3d(
                x=[x0, x1], y=[y0, y1], z=[z0, z1], showlegend=False, hoverinfo='none',
                mode="lines", line=dict(width=0.5, color=colors[0])))
            
            fig.add_trace(go.Scatter3d(
                x=[x1, x2], y=[y1, y2], z=[z1, z2],showlegend=False,  hoverinfo='none',
                mode="lines", line=dict(width=0.5, color=colors[1])))

    fig.update_layout(
        autosize=False,
        width=1500,
        height=1000,
        margin=dict(l=0, r=0, b=0, t=0, pad=0),
        plot_bgcolor=bg_color,
        paper_bgcolor=bg_color,
    )

    fig.update_shapes(dict(xref='x', yref='y'))
    return fig

In [ ]:
fig = plot_3d_conv(dia_table, 1, 14)
fig.show()

In [ ]:
fig = plot_conv(dia_table, idx)
fig.show()